# Local Hyperparameter Optimization

This notebook is similar to the `classifier_hyp` notebook, but it runs the hyperparameter optimization locally instead of on Google Colab. The definitive version is the Google Colab one because we run many more iterations and parameter combinations there due to GPU availability.

In [1]:
import wandb

wandb.login()

wandb: Currently logged in as: e1527193 (flower-classification). Use `wandb login --relogin` to force relogin


True

In [2]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from torchvision import datasets, transforms
from torchvision.models import resnet50, ResNet50_Weights
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler
import numpy as np
import os
import time
import copy
import random
from sklearn import metrics

torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def build_dataset(batch_size):    
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    data_dir = 'plantsdata'
    dataset = datasets.ImageFolder(os.path.join(data_dir))

    # 90/10 split
    train_dataset, test_dataset = random_split(dataset, [0.9, 0.1])

    train_dataset.dataset.transform = data_transforms['train']
    test_dataset.dataset.transform = data_transforms['test']

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                               shuffle=True, num_workers=4)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                             shuffle=True, num_workers=4)

    dataloaders = {'train': train_loader, 'test': test_loader}
    dataset_size = len(dataset)
    dataset_sizes = {'train': len(train_dataset), 'test': len(test_dataset)}
    class_names = dataset.classes

    return (dataloaders, dataset_sizes)

def build_network():
    network = resnet50(weights=ResNet50_Weights.DEFAULT)
    num_ftrs = network.fc.in_features

    # Add linear layer with number of classes
    network.fc = nn.Linear(num_ftrs, 2)

    return network.to(device)

def build_optimizer(network, optimizer, learning_rate, beta_one, beta_two, eps):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate,
                               betas=(beta_one, beta_two),
                               eps=eps)
    return optimizer

def train_epoch(network, loader, optimizer, criterion, scheduler, dataset_sizes):
    network.train()
    running_loss = 0.0
    running_corrects = 0
    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # ➡ Forward pass
        #loss = F.nll_loss(network(data), target)
        with torch.set_grad_enabled(True):
            outputs = network(data)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, target)
        
        #cumu_loss += loss.item()
        
        running_loss += loss.item() * data.size(0)
        running_corrects += torch.sum(preds == target.data)

        # ⬅ Backward pass + weight update
        loss.backward()
        optimizer.step()

        wandb.log({'train/batch_loss': loss.item()})

    scheduler.step()

    epoch_loss = running_loss / dataset_sizes['train']
    epoch_acc = running_corrects.double() / dataset_sizes['train']
    
    return (epoch_loss, epoch_acc)

def test(network, loader, optimizer, criterion, dataset_sizes):
    network.eval()
    confusion = torch.empty([0, 1])
    confusion = confusion.to(device)
    running_loss = 0.0
    test_corrects = 0
    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # ➡ Forward pass
        with torch.set_grad_enabled(False):
            outputs = network(data)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, target)

        running_loss += loss.item() * data.size(0)
        test_corrects += torch.sum(preds == target.data)
        
        confusion = torch.cat((confusion, preds[:, None] / target.data[:, None]))

    tp = torch.sum(confusion == 1).item()
    fp = torch.sum(confusion == float('inf')).item()
    tn = torch.sum(torch.isnan(confusion)).item()
    fn = torch.sum(confusion == 0).item()
    
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f = 2 * ((precision * recall) / (precision + recall))
    
    epoch_loss = running_loss / dataset_sizes['test']
    epoch_acc = test_corrects.double() / dataset_sizes['test']
    
    return (epoch_loss, epoch_acc, precision, recall, f)

In [4]:
def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        (dataloaders, dataset_sizes) = build_dataset(config.batch_size)
        network = build_network()
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate, config.beta_one,
                                    config.beta_two, config.eps)
        criterion = nn.CrossEntropyLoss()
        # Decay LR by a factor of 0.1 every 7 epochs
        exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, config.step_size, config.gamma)

        for epoch in range(config.epochs):            
            (epoch_loss, epoch_acc) = train_epoch(network, dataloaders['train'], optimizer,
                                                  criterion, exp_lr_scheduler,
                                                  dataset_sizes)
            wandb.log({"epoch": epoch, 'train/epoch_loss': epoch_loss, 'train/epoch_acc': epoch_acc})
            
            (test_loss, test_acc, test_precision, test_recall, test_f) = test(network, dataloaders['test'],
                                                                              optimizer, criterion,
                                                                              dataset_sizes)
            wandb.log({'test/epoch_loss': test_loss, 'test/epoch_acc': test_acc,
                       'test/precision': test_precision, 'test/recall': test_recall,
                       'test/f1-score': test_f})

In [5]:
sweep_config = {
    'method': 'random'
}

metric = {
    'name': 'test/epoch_acc',
    'goal': 'maximize'   
}

sweep_config['metric'] = metric

parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
    },
}

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'epochs': {
        'value': 10},
    'batch_size': {
        'values': [4, 8]},
    'learning_rate': {
        'values': [0.1, 0.01, 0.003, 0.001, 0.0003, 0.0001]},
    'step_size': {
        'values': [2, 3, 5, 7]},
    'gamma': {
        'values': [0.1, 0.5]},
    'beta_one': {
        'values': [0.9, 0.99]},
    'beta_two': {
        'values': [0.5, 0.9, 0.99, 0.999]},
    'eps': {
        'values': [1e-08, 0.1, 1]}
})

In [6]:
sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-demo")

Create sweep with ID: eqwnoagh
Sweep URL: https://wandb.ai/flower-classification/pytorch-sweeps-demo/sweeps/eqwnoagh


In [7]:
wandb.agent(sweep_id, train, count=30)

wandb: Agent Starting Run: znahtehx with config:
wandb: 	batch_size: 4
wandb: 	beta_one: 0.9
wandb: 	beta_two: 0.5
wandb: 	epochs: 10
wandb: 	eps: 1
wandb: 	gamma: 0.1
wandb: 	learning_rate: 0.001
wandb: 	optimizer: adam
wandb: 	step_size: 2


epoch,▁▂▃▃▄▅▆▆▇█
test/epoch_acc,▁▂▅▆▅▇▄▇▃█
test/epoch_loss,█▃▂▂▃▂▂▁▂▂
test/f1-score,▁▃▅▆▄▆▄▇▃█
test/precision,▁▁▃▅▆▇▄█▃▇
test/recall,▁▆▇▆▁▄▃▄▃█
train/batch_loss,█▇▆▆▆▆▅▄▄▃▅▅█▆▃█▆▇▂▆▅▅▁▃▆▄▃██▅▆▄▆▅▄▂▂▇▇▆
train/epoch_acc,▁▆▇▇▇███▇█
train/epoch_loss,█▄▂▂▂▁▁▁▂▁
epoch,9
test/epoch_acc,0.85556


wandb: Agent Starting Run: qutqx8ux with config:
wandb: 	batch_size: 4
wandb: 	beta_one: 0.9
wandb: 	beta_two: 0.9
wandb: 	epochs: 10
wandb: 	eps: 1e-08
wandb: 	gamma: 0.1
wandb: 	learning_rate: 0.1
wandb: 	optimizer: sgd
wandb: 	step_size: 2


epoch,▁▂▃▃▄▅▆▆▇█
test/epoch_acc,▁▂▁▆▆█▇▆▃▅
test/epoch_loss,█▅▃▁▁▁▁▂▂▂
test/f1-score,▄▁▆█▇███▆▇
test/precision,▁▃▁▅▆█▆▅▃▄
test/recall,▅▁██▆▆██▆█
train/batch_loss,▄▁▃█▃▇▅▁▄▄▅▃▃▂▂▅▅▂▂▅▃▃▅▄▂▃▃▂▄▃▃▆▅▂▂▄▅▁▂▂
train/epoch_acc,▁▃▄▇▆▇█▇▇█
train/epoch_loss,█▄▃▂▂▁▁▁▁▁
epoch,9
test/epoch_acc,0.7


wandb: Agent Starting Run: 9j8etw77 with config:
wandb: 	batch_size: 8
wandb: 	beta_one: 0.99
wandb: 	beta_two: 0.5
wandb: 	epochs: 10
wandb: 	eps: 0.1
wandb: 	gamma: 0.1
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: adam
wandb: 	step_size: 2


...

Run lfi2onyo errored: OutOfMemoryError('CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 1.95 GiB total capacity; 1.32 GiB already allocated; 1.31 MiB free; 1.33 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: ERROR Run lfi2onyo errored: OutOfMemoryError('CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 1.95 GiB total capacity; 1.32 GiB already allocated; 1.31 MiB free; 1.33 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: Agent Starting Run: 4uvn2tnq with config:
wandb: 	batch_size: 4
wandb: 	beta_one: 0.9
wandb: 	beta_two: 0.5
wandb: 	epochs: 10
wandb: 	eps: 0.1
wandb: 	gamma: 0.5
wandb: 	learning_rate: 0.001
wandb: 	optimizer: adam
wandb: 	step_size: 3


Run 4uvn2tnq errored: OutOfMemoryError('CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 1.95 GiB total capacity; 1.32 GiB already allocated; 1.31 MiB free; 1.33 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: ERROR Run 4uvn2tnq errored: OutOfMemoryError('CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 1.95 GiB total capacity; 1.32 GiB already allocated; 1.31 MiB free; 1.33 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: Agent Starting Run: y4niwbym with config:
wandb: 	batch_size: 4
wandb: 	beta_one: 0.9
wandb: 	beta_two: 0.5
wandb: 	epochs: 10
wandb: 	eps: 1
wandb: 	gamma: 0.5
wandb: 	learning_rate: 0.01
wandb: 	optimizer: sgd
wandb: 	step_size: 2


Run y4niwbym errored: OutOfMemoryError('CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 1.95 GiB total capacity; 1.32 GiB already allocated; 1.31 MiB free; 1.33 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: ERROR Run y4niwbym errored: OutOfMemoryError('CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 1.95 GiB total capacity; 1.32 GiB already allocated; 1.31 MiB free; 1.33 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: Agent Starting Run: hxampiva with config:
wandb: 	batch_size: 8
wandb: 	beta_one: 0.9
wandb: 	beta_two: 0.5
wandb: 	epochs: 10
wandb: 	eps: 0.1
wandb: 	gamma: 0.5
wandb: 	learning_rate: 0.0003
wandb: 	optimizer: sgd
wandb: 	step_size: 3


Run hxampiva errored: OutOfMemoryError('CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 1.95 GiB total capacity; 1.32 GiB already allocated; 1.31 MiB free; 1.33 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: ERROR Run hxampiva errored: OutOfMemoryError('CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 1.95 GiB total capacity; 1.32 GiB already allocated; 1.31 MiB free; 1.33 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: q1v8qruc with config:
wandb: 	batch_size: 8
wandb: 	beta_one: 0.99
wandb: 	beta_two: 0.9
wandb: 	epochs: 10
wandb: 	eps: 1e-08
wandb: 	gamma: 0.1
wandb: 	learning_r

Run q1v8qruc errored: OutOfMemoryError('CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 1.95 GiB total capacity; 1.32 GiB already allocated; 1.31 MiB free; 1.33 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: ERROR Run q1v8qruc errored: OutOfMemoryError('CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 1.95 GiB total capacity; 1.32 GiB already allocated; 1.31 MiB free; 1.33 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
